In [16]:
import pandas as pd
from dotenv import load_dotenv
import tenacity
import requests
import os

In [3]:
%pip install requests



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install pandas


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install tenacity



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
load_dotenv()

False

In [5]:
token1 = os.getenv('token1')
token2 = os.getenv('token2')
token3 = os.getenv('token3')

In [17]:
tokens =  [token1, token2, token3]

In [6]:

owner = 'bclindner'
repo = 'CNMN'

# GitHub API endpoint
api_url = f'https://api.github.com/repos/{owner}/{repo}/contents'


headers = {
    'Authorization': f'token {tokens[0]}',
    'Accept': 'application/vnd.github.v3+json'
}

In [26]:
@tenacity.retry(stop=tenacity.stop_after_attempt(5), wait=tenacity.wait_exponential(multiplier=1, min=4, max=10), 
       retry=tenacity.retry_if_exception_type((requests.ConnectionError, requests.Timeout, requests.HTTPError)))
def get_contents(path=''):
    url = f'{api_url}/{path}'
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def get_all_files_and_folders(path=''):
    contents = get_contents(path)
    files_and_folders = []

    for item in contents:
        if item['type'] == 'file':
            files_and_folders.append(item['path'])
        elif item['type'] == 'dir':
            files_and_folders.append(item['path'])
            files_and_folders.extend(get_all_files_and_folders(item['path']))

    return files_and_folders

def filter_files_by_language(files_and_folders, language_extensions):
    filtered_files = {lang: [] for lang in language_extensions.values()}
    language_counts = {lang: 0 for lang in language_extensions.values()}
    
    for file_path in files_and_folders:
        file_extension = file_path.split('.')[-1]
        if file_extension in language_extensions:
            lang = language_extensions[file_extension]
            filtered_files[lang].append(file_path)
            language_counts[lang] += 1
    
    return filtered_files, language_counts

# Get all files and folders in the repository
try:
    all_files_and_folders = get_all_files_and_folders()
except requests.HTTPError as e:
    print(f"HTTPError: {e}")

# Filter files by the specified programming languages

# Print the programming languages used in the project



Programming languages used in the project:


In [7]:
df = pd.read_csv('issues_event.csv')
df

,repo_name,month
0,bclindner/CNMN,2023-05
1,olmloa/upptime,2023-05
2,SpaceNetLab/StarryNet,2023-05
3,ataitler/RDDL-demo-agent,2023-05
4,cisagov/ScubaGear,2023-05
...,...,...
11995,ubuntu/desktop-snaps,2023-10
11996,usebruno/bruno,2023-10
11997,hlhc/containers,2023-10
11998,jxxghp/MoviePilot,2023-10


In [9]:
df['is_used'] =  0
df

,repo_name,month,is_used
0,bclindner/CNMN,2023-05,0
1,olmloa/upptime,2023-05,0
2,SpaceNetLab/StarryNet,2023-05,0
3,ataitler/RDDL-demo-agent,2023-05,0
4,cisagov/ScubaGear,2023-05,0
...,...,...,...
11995,ubuntu/desktop-snaps,2023-10,0
11996,usebruno/bruno,2023-10,0
11997,hlhc/containers,2023-10,0
11998,jxxghp/MoviePilot,2023-10,0


In [10]:
df['programming_languages'] = ' '

In [11]:
df

,repo_name,month,is_used,programming_languages
0,bclindner/CNMN,2023-05,0,
1,olmloa/upptime,2023-05,0,
2,SpaceNetLab/StarryNet,2023-05,0,
3,ataitler/RDDL-demo-agent,2023-05,0,
4,cisagov/ScubaGear,2023-05,0,
...,...,...,...,...
11995,ubuntu/desktop-snaps,2023-10,0,
11996,usebruno/bruno,2023-10,0,
11997,hlhc/containers,2023-10,0,
11998,jxxghp/MoviePilot,2023-10,0,


In [12]:
first_5_repos = df.head(5)
first_5_repos

,repo_name,month,is_used,programming_languages
0,bclindner/CNMN,2023-05,0,
1,olmloa/upptime,2023-05,0,
2,SpaceNetLab/StarryNet,2023-05,0,
3,ataitler/RDDL-demo-agent,2023-05,0,
4,cisagov/ScubaGear,2023-05,0,


In [15]:
repo_names =  first_5_repos['repo_name'].tolist()
repo_names

['bclindner/CNMN',
 'olmloa/upptime',
 'SpaceNetLab/StarryNet',
 'ataitler/RDDL-demo-agent',
 'cisagov/ScubaGear']

In [17]:
repositories = [{'owner': repo.split('/')[0], 'repo': repo.split('/')[1]} for repo in repo_names]
repositories


[{'owner': 'bclindner', 'repo': 'CNMN'},
 {'owner': 'olmloa', 'repo': 'upptime'},
 {'owner': 'SpaceNetLab', 'repo': 'StarryNet'},
 {'owner': 'ataitler', 'repo': 'RDDL-demo-agent'},
 {'owner': 'cisagov', 'repo': 'ScubaGear'}]

In [4]:
repos_hundred_test = pd.read_csv('hundred_twenty_repos.csv')
repos_hundred_test
repos_hundred_test['languages'] = ''
repos_hundred_test['stars_count'] = ''
repos_hundred_test['forks_count']= ''
repos_hundred_test['resolution_time_hours'] = ''

repos_hundred_test_cleaned = repos_hundred_test.drop_duplicates(subset="repo_name")



,repo_name,month,languages,stars_count,forks_count,resolution_time_hours
0,BitCrackers/AmongUsMenu,2023-01,,,,
1,EmirBurgosPaz/interfaz_arduino,2023-01,,,,
2,Filigrani/SkyCoop,2023-01,,,,
3,Q309166/-,2023-01,,,,
4,apache/age,2023-01,,,,
...,...,...,...,...,...,...
115,department-of-veterans-affairs/va.gov-team,2023-12,,,,
116,element-hq/synapse,2023-12,,,,
117,ghscr/ghscription,2023-12,,,,
118,h4sh5/pypi-auto-scanner,2023-12,,,,


In [5]:
repos_hundred_test_cleaned = repos_hundred_test.drop_duplicates(subset="repo_name")
repos_hundred_test_cleaned
#114 rows left. 

,repo_name,month,languages,stars_count,forks_count,resolution_time_hours
0,BitCrackers/AmongUsMenu,2023-01,,,,
1,EmirBurgosPaz/interfaz_arduino,2023-01,,,,
2,Filigrani/SkyCoop,2023-01,,,,
3,Q309166/-,2023-01,,,,
4,apache/age,2023-01,,,,
...,...,...,...,...,...,...
115,department-of-veterans-affairs/va.gov-team,2023-12,,,,
116,element-hq/synapse,2023-12,,,,
117,ghscr/ghscription,2023-12,,,,
118,h4sh5/pypi-auto-scanner,2023-12,,,,


In [75]:
import requests
from datetime import datetime


# Function to get repository details and issues using GraphQL
def get_repo_details_and_issues(owner, repo, token):
    headers = {
    'Authorization': f'bearer {token}'
}
    print(token)
    url = "https://api.github.com/graphql"
    query_template = f"""
    {{
      repository(owner: "{owner}", name: "{repo}") {{
        stargazerCount
        forkCount
        languages(first: 10) {{
          edges {{
            size
            node {{
              name
            }}
          }}
        }}
        issues(first: 100, states: CLOSED) {{
          edges {{
            node {{
              createdAt
              closedAt
            }}
          }}
        }}
      }}
    }}
    """
    response = requests.post(url, headers=headers, json={"query": query_template})
    response.raise_for_status()

    if response.status_code != 200:
        print(f"Failed to fetch data for {owner}/{repo}: {response.status_code}")
        return None
    response_data = response.json()
    if 'errors' in response_data:
        print(f"Errors in response for {owner}/{repo}: {response_data['errors']}")
        return None
    
    return response_data

# Function to calculate the average resolution time for issues
def calculate_average_resolution_time(issues):
    total_resolution_time = 0
    resolved_issue_count = 0

    for issue in issues:
        if issue['closedAt'] is not None:
          created_at = datetime.strptime(issue['createdAt'], '%Y-%m-%dT%H:%M:%SZ')
          closed_at = datetime.strptime(issue['closedAt'], '%Y-%m-%dT%H:%M:%SZ')
          resolution_time = closed_at - created_at
          total_resolution_time += resolution_time.total_seconds()
          resolved_issue_count += 1

    if resolved_issue_count == 0:
        return 0

    average_resolution_time = total_resolution_time / resolved_issue_count
    return average_resolution_time / 3600  # return in hours


def play_around():
  repo_info = get_repo_details_and_issues(owner, repo)

  # Extract repository details
  stargazers_count = repo_info['data']['repository']['stargazerCount']
  fork_count = repo_info['data']['repository']['forkCount']
  languages = repo_info['data']['repository']['languages']['edges']

  # Extract issues and calculate average resolution time
  issues = [edge['node'] for edge in repo_info['data']['repository']['issues']['edges']]
  average_time = calculate_average_resolution_time(issues)

  # Print the results
  print(f"Average issue resolution time: {average_time:.2f} hours")
  print(f"Stars: {stargazers_count}")
  print(f"Forks: {fork_count}")
  print("Languages:")
  for language in languages:
      print(f"  {language['node']['name']}: {language['size']} bytes")


In [13]:
for index, row in repos_hundred_test_cleaned.iterrows():
    if row['repo_name']:
        owner_repo = row['repo_name'].split('/')
        if len(owner_repo) == 2:
            owner, repo = owner_repo
            repo_info = get_repo_details_and_issues(owner, repo)

            if repo_info and repo_info.get('data') and repo_info['data'].get('repository'):
                print(repo_info)
                # Extract repository details
                stargazers_count = repo_info['data']['repository']['stargazerCount']
                fork_count = repo_info['data']['repository']['forkCount']
                languages = repo_info['data']['repository']['languages']['edges']

                # Extract issues and calculate average resolution time
                issues = [edge['node'] for edge in repo_info['data']['repository']['issues']['edges']]
                average_time = calculate_average_resolution_time(issues)

                # Update the dataframe
                repos_hundred_test_cleaned.at[index, 'stars_count'] = stargazers_count
                repos_hundred_test_cleaned.at[index, 'forks_count'] = fork_count
                repos_hundred_test_cleaned.at[index, 'languages'] = ', '.join([language['node']['name'] for language in languages])
                repos_hundred_test_cleaned.at[index, 'resolution_time_hours'] = average_time
            else:
                print(f"Repository {owner}/{repo} not found or API response is malformed.")
        else:
            print(f"Invalid repository name format: {row['repo_name']}")


repos_hundred_test_cleaned

{'data': {'repository': {'stargazerCount': 459, 'forkCount': 913, 'languages': {'edges': [{'size': 577182, 'node': {'name': 'C'}}, {'size': 3503842, 'node': {'name': 'C++'}}]}, 'issues': {'edges': [{'node': {'createdAt': '2020-11-28T10:08:07Z', 'closedAt': '2020-12-04T16:08:17Z'}}, {'node': {'createdAt': '2020-11-28T14:26:27Z', 'closedAt': '2020-11-28T16:38:33Z'}}, {'node': {'createdAt': '2020-11-28T17:17:15Z', 'closedAt': '2020-12-03T12:37:18Z'}}, {'node': {'createdAt': '2020-11-28T17:30:51Z', 'closedAt': '2020-11-28T19:40:52Z'}}, {'node': {'createdAt': '2020-11-28T17:38:01Z', 'closedAt': '2020-11-28T17:47:24Z'}}, {'node': {'createdAt': '2020-11-28T17:44:44Z', 'closedAt': '2020-11-29T14:26:42Z'}}, {'node': {'createdAt': '2020-11-28T21:30:49Z', 'closedAt': '2020-11-29T00:20:38Z'}}, {'node': {'createdAt': '2020-11-29T10:38:27Z', 'closedAt': '2021-03-21T22:43:44Z'}}, {'node': {'createdAt': '2020-11-29T12:49:40Z', 'closedAt': '2020-11-30T00:44:28Z'}}, {'node': {'createdAt': '2020-11-29T21

,repo_name,month,languages,stars_count,forks_count,resolution_time_hours
0,BitCrackers/AmongUsMenu,2023-01,"C, C++",459,913,396.961214
1,EmirBurgosPaz/interfaz_arduino,2023-01,C++,0,0,0
2,Filigrani/SkyCoop,2023-01,C#,154,24,1236.49121
3,Q309166/-,2023-01,,1,0,0
4,apache/age,2023-01,"C, Makefile, PLpgSQL, Roff, Lex, Yacc, Shell, ...",2829,399,2110.030542
...,...,...,...,...,...,...
115,department-of-veterans-affairs/va.gov-team,2023-12,"JavaScript, Ruby, HTML, CSS, Jupyter Notebook,...",277,194,7420.032578
116,element-hq/synapse,2023-12,"Shell, JavaScript, Python, Perl, Lua, Rust, CS...",970,110,3.702775
117,ghscr/ghscription,2023-12,,,,
118,h4sh5/pypi-auto-scanner,2023-12,"Python, Shell, Go",26,7,71.142344


In [14]:
repos_hundred_test_cleaned.to_csv("hundred_repos_data.csv", index=False)

In [16]:
from bs4 import BeautifulSoup

url = "https://top1000repos.com/"

# Send a GET request to the website
response = requests.get(url)
response.raise_for_status()  # Raise an exception for HTTP errors

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Finding all relevant div elements containing the repository names
container_div = soup.find('div', class_='md:w-3/4')  # Adjust this class based on the structure
main_div = soup.find('main', class_ = 'px-3 sm:px-4 lg:px-5 md:flex md:gap-2 lg:max-w-5xl lg:mx-auto grow w-full')
# print(main_div)
repo_names = []

repos_div = soup.find('div', class_='md:w-3/4')
print(repos_div)

repo_name_div = soup.find('div', class_='flex gap-2')
print(repo_name_div)

# Loop through each nested div and extract the repository name
if container_div:
    repo_divs = container_div.find_all('div', class_='flex false -ml-3 md:-ml-2')
    print(repo_divs)
    for repo_div in repo_divs:
        repo_name_element = repo_div.find('a')  # Assuming the repo name is within an 'a' tag
        if repo_name_element:
            repo_name = repo_name_element.text.strip()
            repo_names.append(repo_name)
else:
    print("Target container div not found")

# Display the first 10 repository names
for name in repo_names[:10]:
    print(name)


<div class="md:w-3/4">
<nav class="hidden md:block text-sm font-medium text-center border-b border-gray-200 sticky top-0 z-10 bg-white"><ul class="flex flex-wrap -mb-px"><li class="mr-2"><a aria-current="page" class="inline-block p-4 border-b-2 text-gray-700 border-blue-500" href="#">Browse (1000)</a></li>
<li class="mr-2"><a class="border-transparent inline-block p-4 border-b-2 text-gray-500 hover:text-gray-600 hover:border-gray-300" href="#">Hidden (0)</a></li></ul></nav>
<div class="flex flex-col gap-6 py-5">
<p></p></div></div>
None
[]


In [19]:
with open('snippets.html', 'r') as file:
  html_text = file.read()

In [23]:

soup = BeautifulSoup(html_text, 'html.parser')

In [25]:
links = soup.find_all('a')
extracted_links = [link.get('href') for link in links]
print(extracted_links)

['https://github.com/freeCodeCamp/freeCodeCamp', 'https://github.com/freeCodeCamp/freeCodeCamp/pulls?q=is%3Apr+sort%3Areactions-%2B1-desc+is%3Aclosed+closed%3A%3E2023-07-06', 'https://github.com/freeCodeCamp/freeCodeCamp/issues?q=is%3Aissue+sort%3Areactions-%2B1-desc+is%3Aclosed+closed%3A%3E2023-07-06', 'https://github.com/EbookFoundation/free-programming-books', 'https://github.com/EbookFoundation/free-programming-books/pulls?q=is%3Apr+sort%3Areactions-%2B1-desc+is%3Aclosed+closed%3A%3E2023-07-06', 'https://github.com/EbookFoundation/free-programming-books/issues?q=is%3Aissue+sort%3Areactions-%2B1-desc+is%3Aclosed+closed%3A%3E2023-07-06', 'https://github.com/sindresorhus/awesome', 'https://github.com/sindresorhus/awesome/pulls?q=is%3Apr+sort%3Areactions-%2B1-desc+is%3Aclosed+closed%3A%3E2023-07-06', 'https://github.com/sindresorhus/awesome/issues?q=is%3Aissue+sort%3Areactions-%2B1-desc+is%3Aclosed+closed%3A%3E2023-07-06', 'https://github.com/jwasham/coding-interview-university', 'http

In [28]:
data = []

for link in extracted_links:
    data.append({
      'repo_name': link  
  })

df_top1000 = pd.DataFrame(data)
df_top1000

,repo_name
0,https://github.com/freeCodeCamp/freeCodeCamp
1,https://github.com/freeCodeCamp/freeCodeCamp/p...
2,https://github.com/freeCodeCamp/freeCodeCamp/i...
3,https://github.com/EbookFoundation/free-progra...
4,https://github.com/EbookFoundation/free-progra...
...,...
2952,https://github.com/heartcombo/devise/pulls?q=i...
2953,https://github.com/heartcombo/devise/issues?q=...
2954,https://github.com/rwf2/Rocket
2955,https://github.com/rwf2/Rocket/pulls?q=is%3Apr...


In [29]:
df_top1000['repo_name'] = df_top1000['repo_name'].str.replace('https://github.com/', '', regex=True)


In [30]:
df_top1000

,repo_name
0,freeCodeCamp/freeCodeCamp
1,freeCodeCamp/freeCodeCamp/pulls?q=is%3Apr+sort...
2,freeCodeCamp/freeCodeCamp/issues?q=is%3Aissue+...
3,EbookFoundation/free-programming-books
4,EbookFoundation/free-programming-books/pulls?q...
...,...
2952,heartcombo/devise/pulls?q=is%3Apr+sort%3Areact...
2953,heartcombo/devise/issues?q=is%3Aissue+sort%3Ar...
2954,rwf2/Rocket
2955,rwf2/Rocket/pulls?q=is%3Apr+sort%3Areactions-%...


In [31]:
filtered_df = df_top1000[~df_top1000['repo_name'].str.contains('pulls|issues', case=False, regex=True)]

# Print the filtered dataframe
filtered_df

,repo_name
0,freeCodeCamp/freeCodeCamp
3,EbookFoundation/free-programming-books
6,sindresorhus/awesome
9,jwasham/coding-interview-university
12,public-apis/public-apis
...,...
2942,VundleVim/Vundle.vim
2945,ai/nanoid
2948,rome/tools
2951,heartcombo/devise


In [32]:
filtered_df['languages'] = ''
filtered_df['stars_count'] = ''
filtered_df['forks_count']= ''
filtered_df['resolution_time_hours'] = ''

/var/folders/1t/4fq5wg391gjfvgg3qdyqfh2m0000gn/T/ipykernel_14254/172228104.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['languages'] = ''
/var/folders/1t/4fq5wg391gjfvgg3qdyqfh2m0000gn/T/ipykernel_14254/172228104.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['stars_count'] = ''
/var/folders/1t/4fq5wg391gjfvgg3qdyqfh2m0000gn/T/ipykernel_14254/172228104.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [55]:
filtered_df

,repo_name,languages,stars_count,forks_count,resolution_time_hours
0,freeCodeCamp/freeCodeCamp,,,,
3,EbookFoundation/free-programming-books,,,,
6,sindresorhus/awesome,,,,
9,jwasham/coding-interview-university,,,,
12,public-apis/public-apis,,,,
...,...,...,...,...,...
2942,VundleVim/Vundle.vim,,,,
2945,ai/nanoid,,,,
2948,rome/tools,,,,
2951,heartcombo/devise,,,,


In [20]:
current_token_index  = 0
print(f"owner_repo length: {len(owner_repo)}, tokens: {tokens}, current_token_index: {current_token_index}")

for index, row in filtered_df.iterrows():
    
    if row['repo_name']:
        owner_repo = row['repo_name'].split('/')
        if len(owner_repo) == 2:
            owner, repo = owner_repo
            repo_info = get_repo_details_and_issues(owner, repo,tokens[current_token_index])
            print(repo_info, type(repo_info))


            if repo_info and repo_info.get('data') and repo_info['data'].get('repository'):
                print(repo_info)
                # Extract repository details
                stargazers_count = repo_info['data']['repository']['stargazerCount']
                fork_count = repo_info['data']['repository']['forkCount']
                languages = repo_info['data']['repository']['languages']['edges']

                # Extract issues and calculate average resolution time
                issues = [edge['node'] for edge in repo_info['data']['repository']['issues']['edges']]
                average_time = calculate_average_resolution_time(issues)

                # Update the dataframe
                filtered_df.at[index, 'stars_count'] = stargazers_count
                filtered_df.at[index, 'forks_count'] = fork_count
                filtered_df.at[index, 'languages'] = ', '.join([language['node']['name'] for language in languages])
                filtered_df.at[index, 'resolution_time_hours'] = average_time
            else:
                print(f"Repository {owner}/{repo} not found or API response is malformed.")
        else:
            print(f"Invalid repository name format: {row['repo_name']}")
    current_token_index = (current_token_index + 1) % len(tokens)


filtered_df

NameError: name 'owner_repo' is not defined

In [66]:
filtered_df

,repo_name,languages,stars_count,forks_count,resolution_time_hours
0,freeCodeCamp/freeCodeCamp,"JavaScript, CSS, HTML, Shell, Less, EJS, Docke...",392612,36105,162.064075
3,EbookFoundation/free-programming-books,,326257,60530,426.159211
6,sindresorhus/awesome,,309686,27008,3234.542911
9,jwasham/coding-interview-university,,297894,74980,4571.094631
12,public-apis/public-apis,"Shell, Python",297846,32286,1146.081406
...,...,...,...,...,...
2942,VundleVim/Vundle.vim,,,,
2945,ai/nanoid,,,,
2948,rome/tools,,,,
2951,heartcombo/devise,,,,


In [67]:
filtered_df.to_csv("thousand.csv", index=False)


In [70]:
filtered_df

,repo_name,languages,stars_count,forks_count,resolution_time_hours
0,freeCodeCamp/freeCodeCamp,"JavaScript, CSS, HTML, Shell, Less, EJS, Docke...",392612,36105,162.064075
3,EbookFoundation/free-programming-books,,326257,60530,426.159211
6,sindresorhus/awesome,,309686,27008,3234.542911
9,jwasham/coding-interview-university,,297894,74980,4571.094631
12,public-apis/public-apis,"Shell, Python",297846,32286,1146.081406
...,...,...,...,...,...
2942,VundleVim/Vundle.vim,,,,
2945,ai/nanoid,,,,
2948,rome/tools,,,,
2951,heartcombo/devise,,,,


In [72]:
# Ensure there is a row with repo_name 'ggreer/the_silver_searcher'
if not filtered_df[filtered_df['repo_name'] == "ggreer/the_silver_searcher"].empty:
    # Find the first occurrence of the row where "repo_name" is "ggreer/the_silver_searcher"
    split_index = filtered_df[filtered_df['repo_name'] == "ggreer/the_silver_searcher"].index[0]
    
    # Split the DataFrame
    df1 = filtered_df.loc[:split_index - 1]
    df2 = filtered_df.loc[split_index:]
    
    
else:
    print("No row with repo_name 'ggreer/the_silver_searcher' found.")


In [19]:
current_token_index  = 0

for index, row in df2.iterrows():
    
    if row['repo_name']:
        owner_repo = row['repo_name'].split('/')
        if len(owner_repo) == 2:
            owner, repo = owner_repo
            repo_info = get_repo_details_and_issues(owner, repo,tokens[current_token_index])
            print(repo_info, type(repo_info))


            if repo_info and repo_info.get('data') and repo_info['data'].get('repository'):
                print(repo_info)
                # Extract repository details
                stargazers_count = repo_info['data']['repository']['stargazerCount']
                fork_count = repo_info['data']['repository']['forkCount']
                languages = repo_info['data']['repository']['languages']['edges']

                # Extract issues and calculate average resolution time
                issues = [edge['node'] for edge in repo_info['data']['repository']['issues']['edges']]
                average_time = calculate_average_resolution_time(issues)

                # Update the dataframe
                df2.at[index, 'stars_count'] = stargazers_count
                df2.at[index, 'forks_count'] = fork_count
                df2.at[index, 'languages'] = ', '.join([language['node']['name'] for language in languages])
                df2.at[index, 'resolution_time_hours'] = average_time
            else:
                print(f"Repository {owner}/{repo} not found or API response is malformed.")
        else:
            print(f"Invalid repository name format: {row['repo_name']}")
    current_token_index = (current_token_index + 1) % len(tokens)


df2

NameError: name 'df2' is not defined

In [77]:
df1

,repo_name,languages,stars_count,forks_count,resolution_time_hours
0,freeCodeCamp/freeCodeCamp,"JavaScript, CSS, HTML, Shell, Less, EJS, Docke...",392612,36105,162.064075
3,EbookFoundation/free-programming-books,,326257,60530,426.159211
6,sindresorhus/awesome,,309686,27008,3234.542911
9,jwasham/coding-interview-university,,297894,74980,4571.094631
12,public-apis/public-apis,"Shell, Python",297846,32286,1146.081406
...,...,...,...,...,...
2479,rstacruz/nprogress,"HTML, CSS, JavaScript",25969,1810,5276.43601
2482,dnSpy/dnSpy,"C#, Batchfile, PowerShell, Visual Basic .NET",25962,4997,0
2484,iawia002/lux,"Go, JavaScript",25951,2876,803.119803
2487,makeplane/plane,"TypeScript, JavaScript, CSS, Shell, Procfile, ...",25934,1403,1961.373225


In [79]:
combined_df = pd.concat([df1, df2])

combined_df

,repo_name,languages,stars_count,forks_count,resolution_time_hours
0,freeCodeCamp/freeCodeCamp,"JavaScript, CSS, HTML, Shell, Less, EJS, Docke...",392612,36105,162.064075
3,EbookFoundation/free-programming-books,,326257,60530,426.159211
6,sindresorhus/awesome,,309686,27008,3234.542911
9,jwasham/coding-interview-university,,297894,74980,4571.094631
12,public-apis/public-apis,"Shell, Python",297846,32286,1146.081406
...,...,...,...,...,...
2942,VundleVim/Vundle.vim,"Erlang, Vim Script",23867,2565,5379.637931
2945,ai/nanoid,"JavaScript, HTML",23851,778,800.955917
2948,rome/tools,"JavaScript, SCSS, Rust, TypeScript, CSS, Shell...",23831,666,804.737856
2951,heartcombo/devise,"Ruby, HTML",23827,5535,85.822956


In [80]:
combined_df.to_csv('topthousandrepos.csv', index=False)

In [11]:
df_test1 = pd.read_csv('topthousandrepos.csv')
df_test1

,repo_name,languages,stars_count,forks_count,resolution_time_hours
0,freeCodeCamp/freeCodeCamp,"JavaScript, CSS, HTML, Shell, Less, EJS, Docke...",392612,36105,162.064075
1,EbookFoundation/free-programming-books,NaN,326257,60530,426.159211
2,sindresorhus/awesome,NaN,309686,27008,3234.542911
3,jwasham/coding-interview-university,NaN,297894,74980,4571.094631
4,public-apis/public-apis,"Shell, Python",297846,32286,1146.081406
...,...,...,...,...,...
995,VundleVim/Vundle.vim,"Erlang, Vim Script",23867,2565,5379.637931
996,ai/nanoid,"JavaScript, HTML",23851,778,800.955917
997,rome/tools,"JavaScript, SCSS, Rust, TypeScript, CSS, Shell...",23831,666,804.737856
998,heartcombo/devise,"Ruby, HTML",23827,5535,85.822956


In [13]:
df_test1['resolution_time_days'] = df_test1['resolution_time_hours'].apply(
    lambda x: x / 24 if x > 24 else x
)

In [15]:
df_test1['resolution_time_days'] = df_test1['resolution_time_days'].round(1)
df_test1


,repo_name,languages,stars_count,forks_count,resolution_time_hours,resolution_time_days
0,freeCodeCamp/freeCodeCamp,"JavaScript, CSS, HTML, Shell, Less, EJS, Docke...",392612,36105,162.064075,6.8
1,EbookFoundation/free-programming-books,NaN,326257,60530,426.159211,17.8
2,sindresorhus/awesome,NaN,309686,27008,3234.542911,134.8
3,jwasham/coding-interview-university,NaN,297894,74980,4571.094631,190.5
4,public-apis/public-apis,"Shell, Python",297846,32286,1146.081406,47.8
...,...,...,...,...,...,...
995,VundleVim/Vundle.vim,"Erlang, Vim Script",23867,2565,5379.637931,224.2
996,ai/nanoid,"JavaScript, HTML",23851,778,800.955917,33.4
997,rome/tools,"JavaScript, SCSS, Rust, TypeScript, CSS, Shell...",23831,666,804.737856,33.5
998,heartcombo/devise,"Ruby, HTML",23827,5535,85.822956,3.6
